## Практическая работа №4
##### Сиразетдинов Рустем (Вариант - 20)

1.0 Подготовка датасета для последующего решения задачи

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, learning_curve, ShuffleSplit, cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score, roc_auc_score, roc_curve
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.pipeline import make_pipeline
# from pandas_profiling import ProfileReport
from sklearn.impute import KNNImputer
from sklearn.ensemble import RandomForestClassifier
import warnings
warnings.filterwarnings("ignore")
# import lightgbm as lgb
from sklearn import svm
from scipy.special import boxcox1p
from scipy.stats import boxcox_normmax
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 100)

In [2]:
# import pandas
# import numpy as np
# import io

data = pd.read_csv('C:\\Users\\averu\\Documents\\git_local\\programming-practice\\IDA-practice-4\\BankChurners.csv', index_col='CLIENTNUM')

bank_df = data.loc[:, data.columns.isin(['Marital_Status', 'Credit_Limit', 'Customer_Age', 'Gender', 'Education_Level', 'Income_Category', 'Months_on_book'])]

bank_df = bank_df.dropna()

bank_df['Gender']          = np.where(bank_df['Gender'] == 'M', 1, 0)

bank_df['Education_Level'] = np.where(bank_df['Education_Level'] == 'Unknown' , 1 , bank_df['Education_Level'])
bank_df['Education_Level'] = np.where(bank_df['Education_Level'] == 'Uneducated' , 1 , bank_df['Education_Level'])
bank_df['Education_Level'] = np.where(bank_df['Education_Level'] == 'High School' , 2 , bank_df['Education_Level'])
bank_df['Education_Level'] = np.where(bank_df['Education_Level'] == 'Graduate' , 3 , bank_df['Education_Level'])
bank_df['Education_Level'] = np.where(bank_df['Education_Level'] == 'College', 3 , bank_df['Education_Level'])
bank_df['Education_Level'] = np.where(bank_df['Education_Level'] == 'Post-Graduate' , 4 , bank_df['Education_Level'])
bank_df['Education_Level'] = np.where(bank_df['Education_Level'] == 'Doctorate' , 4 , bank_df['Education_Level'])

bank_df['Income_Category'] = np.where(bank_df['Income_Category'] == 'Unknown' , 0, bank_df['Income_Category'])
bank_df['Income_Category'] = np.where(bank_df['Income_Category'] == 'Less than $40K' , 1, bank_df['Income_Category'])
bank_df['Income_Category'] = np.where(bank_df['Income_Category'] == '$40K - $60K' , 2, bank_df['Income_Category'])
bank_df['Income_Category'] = np.where(bank_df['Income_Category'] == '$60K - $80K' , 3, bank_df['Income_Category'])
bank_df['Income_Category'] = np.where(bank_df['Income_Category'] == '$80K - $120K' , 4, bank_df['Income_Category'])
bank_df['Income_Category'] = np.where(bank_df['Income_Category'] == '$120K +' , 5, bank_df['Income_Category'])

bank_df['Credit_Limit']    = np.where([ (float(x) > 10000) for x in bank_df['Credit_Limit'] ] , 1 , 0)
# bank_df['Credit_Limit']    = [ (float(x) > 10000) for x in bank_df['Credit_Limit'] ]

bank_df['Marital_Status'] = np.where(bank_df['Marital_Status'] == 'Married' , False, bank_df['Marital_Status'])
bank_df['Marital_Status'] = np.where(bank_df['Marital_Status'] == 'Single' , True, bank_df['Marital_Status'])
bank_df['Marital_Status'] = np.where(bank_df['Marital_Status'] == 'Divorced' , True, bank_df['Marital_Status'])
bank_df['Marital_Status'] = np.where(bank_df['Marital_Status'] == 'Unknown' , True, bank_df['Marital_Status'])

M_status = bank_df.loc[:, bank_df.columns.isin(['Marital_Status'])]
# settings = bank_df.loc[:, bank_df.columns.isin(['Credit_Limit', 'Customer_Age', 'Gender', 'Education_Level', 'Income_Category' , 'Months_on_book'])]
settings = bank_df.loc[:, bank_df.columns.isin(['Customer_Age', 'Gender', 'Education_Level', 'Income_Category' , 'Months_on_book'])]

settings

,Customer_Age,Gender,Education_Level,Income_Category,Months_on_book
CLIENTNUM,,,,,
768805383,45,1,2,3,39
818770008,49,0,3,1,44
713982108,51,1,3,4,36
769911858,40,0,2,1,34
709106358,40,1,1,3,21
...,...,...,...,...,...
772366833,50,1,3,2,40
710638233,41,1,1,2,25
716506083,44,0,2,1,36


In [3]:
# Standardize
std = StandardScaler()
std_df = pd.DataFrame(std.fit_transform(bank_df.iloc[:,1:]), 
                      index = bank_df.index, 
                      columns = bank_df.columns[1:])
std_bank_df = pd.concat([bank_df.iloc[:,0], std_df], axis=1)

In [6]:
# Разделение выборки на обучающаю и тестовую
# x_train, x_test, y_train, y_test = train_test_split(settings, M_status, test_size=0.35, random_state=2020)
# SVM
X_train, X_test, y_train, y_test = train_test_split(bank_df.iloc[:,1:], bank_df.iloc[:,0], test_size=0.25, random_state=42)
svm_clf = svm.SVC()

In [7]:
print( "accuracy:"+str(np.average(cross_val_score(svm_clf, X_test, y_test, scoring=  'accuracy'))))
print(       "f1:"+str(np.average(cross_val_score(svm_clf, X_test, y_test, scoring=  'f1'))))
print("precision:"+str(np.average(cross_val_score(svm_clf, X_test, y_test, scoring=  'precision'))))
print(   "recall:"+str(np.average(cross_val_score(svm_clf, X_test, y_test, scoring=  'recall'))))


accuracy:0.09557577316774642
f1:nan
precision:nan
recall:nan


1.1 Подключение необходимых библиотек

In [70]:
# from sklearn.linear_model import SGDClassifier
# from sklearn.linear_model import LogisticRegression
# from sklearn.model_selection import train_test_split
# from sklearn import datasets


1.2 Построение классификатора SVM

In [ ]:
clf = SGDClassifier(loss="log", penalty="l2")
# y_train__ = y_train.ravel()
clf = clf.fit(x_train, y_train)

In [ ]:
y_pred = clf.predict(x_test)
clf.score(x_test, y_test)

In [ ]:
clf2 = LogisticRegression(random_state=0, solver='lbfgs', multi_class='multinomial')
clf2.fit(x_train, y_train)

In [ ]:
y_pred = clf2.predict(x_test)
clf2.score(x_test, y_test)

1.3 Оценка точности построенного классификатора с помощью метрик: precision, recall и F1

In [ ]:
from sklearn.model_selection import cross_val_score
print("accuracy:"+str(np.average(cross_val_score(clf, x_test, y_test, scoring='accuracy'))))
print("f1:"+str(np.average(cross_val_score(clf, x_test, y_test, scoring='f1'))))
print("precision:"+str(np.average(cross_val_score(clf, x_test, y_test, scoring='precision'))))
print("recall:"+str(np.average(cross_val_score(clf, x_test, y_test, scoring='recall'))))

2.1 Построение классификатора типа "Random Forest" для нашей задачи классификации

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
x_train, x_validation, y_train, y_validation = train_test_split(settings, M_status, test_size=.35, random_state=1)

tree = DecisionTreeClassifier(random_state=241, max_depth=5)  # наилучшая максимальная глубина
tree = tree.fit(x_train, y_train)
# for i in range(3, 30):
#     tree = DecisionTreeClassifier(random_state=241, max_depth = i)
#     tree = tree.fit(x_train, y_train)
#     print(str(i) + ": " + str(tree.score(x_validation, y_validation)))

In [ ]:
from sklearn.metrics import accuracy_score
validation_acc = accuracy_score(y_validation, tree.predict(x_validation))
print('Точность на валидации:', validation_acc)


from sklearn.model_selection import cross_val_score
print("accuracy:"+str(np.average(cross_val_score(tree, x_validation, y_validation, scoring='accuracy'))))
print("f1:"+str(np.average(cross_val_score(tree, x_validation, y_validation, scoring='f1'))))
print("precision:"+str(np.average(cross_val_score(tree, x_validation, y_validation, scoring='precision'))))
print("recall:"+str(np.average(cross_val_score(tree, x_validation, y_validation, scoring='recall'))))

In [ ]:
from sklearn.ensemble import RandomForestClassifier 
from sklearn.model_selection import GridSearchCV

param_grid = { 'n_estimators': [200, 300, 400],'max_features': ['auto'], 'max_depth' : [5], 'criterion' :['gini']}

RFC = GridSearchCV(estimator=RandomForestClassifier(), param_grid=param_grid, cv= 5, refit = True) 
RFC.fit(x_train, y_train)


1.3 Оценка точности построенного классификатора с помощью метрик: precision, recall и F1

In [ ]:
print("accuracy:"+str(np.average(cross_val_score(RFC.best_estimator_, x_validation, y_validation, scoring='accuracy'))))
print("f1:"+str(np.average(cross_val_score(RFC.best_estimator_, x_validation, y_validation, scoring='f1'))))
print("precision:"+str(np.average(RFC(tree.best_estimator, x_validation, y_validation, scoring='precision'))))
print("recall:"+str(np.average(RFC(tree.best_estimator, x_validation, y_validation, scoring='recall'))))